##**Timeline**
- Looked at previous dashboard code to learn what was used and what is needed (libraries and hardware)
- Made an official Formula SAE GitHub account to contain and log changes we made to any code that we used
- Made a mock-up tachometer to experiment on how to improve it and get more experience with working with Arduino
- Experimented with start-up animations for the tachometer
- Modified previous dashboard code to omit code that is no longer needed for the new dashboard (such as: 4 digit 14-segment LED displays, tachometer LED-strip clear functions)
- Experimented with a PWM driver for controlling brightness on the dashboard display and drive a 7-segment RGB 1" display for the gear position indicator
- Tested 2 warning LEDs for engine temperature and engine oil pressure with PWM driver
- Acquired 7-segment RGB 1" display from Adafruit. Soldered 7 wires connected to PWM driver to test functionality
- Acquired 7-segment RGB 1" display with NeoPixel support from RGBDigit. Simplifies amount of pins required and code (only 3 pins and uses NeoPixel library). No need for PWM driver to control 7-segment display.
- Made a mock-up gear position shifter with color matching to tachometer
- Updated dashboard code to support PWM driver and 7-segment display. Rough draft ready for testing
- Acquired main arduino board, CAN BUS shield, and custom power PCB shield for testing new dashboard code. Made prototype of all wiring to test with dynamometer. See **Dyno To Do Procedure**

##**Hardware**
- Arduino Uno
- Custom PCB power shield
- 2 x 10k ohm resistor
- 1 x 330 ohm resistor

- [CAN-BUS Shield v1.2](https://www.seeedstudio.com/CAN-BUS-Shield-V1.2-p-2256.html)
  - Libraries: mcp_can.h, SPI.h
  - Requires digital pins 9-13

- 2 x [Adafruit NeoPixel Stick 8 x 5050 RGB LED](https://www.adafruit.com/product/1426)
  - Libraries: Adafruit_NeoPixel.h library
  - Operational Frequency: 800 kHz datastream
  - Can use any digital output pin
  - 300-500 ohm resistor on data line
  
- [Adafruit 12-chanel 16-bit PWM LED driver SPI](https://www.adafruit.com/product/1455)
  - Libraries: Adafruit_TLC59711.h, SPI.h
  - 9600 baud rate
  - Any digital pin can be used for the SPI data output and SPI clock
  - 12V on V+
  
- [RGBDigit 1" 7-segment RGB Display](https://www.rgbdigit.com/shop/index.php?id_product=8&controller=product&id_lang=1)
  - Favored using this version for its simplicity
  - Built in 330 ohm resistor on data pin

###Code Documentation
Dashboard Code

##Libraries

In [ ]:
// Libraries:
#include <mcp_can.h>
#include <SPI.h>
#include <Adafruit_NeoPixel.h>
#include <Adafruit_TLC59711.h>

- mcp_can: used for allowing the CAN-BUS Shield to communicate between the CAN line and the arduino board. Publicly available from Seeed Studio.
- SPI: library for communication between several hardware. Included with Arduino.
- Adafruit_Neopixel: all NeoPixels rely on this library to function (LED strip and 7-segment display). Publicly available from Adafruit.
- Adafruit_TLC59711: handles communication with the TLC59711 PWM driver. Publicly available from Adafruit.

##Defines and Initializations
Non-variable and final values should be defined here as #defines so it can be changed easily if need be and can't be changed while running the code.

In [ ]:
// LED neopixel strip parameters
#define SPI_CS_PIN 9                                  // Declares D9 as CS for Seeed's CAN-BUS Shield
#define stripLength 16                                // NeoPixel LED strip length of 16
#define stripDataPin 5                                // Declares D5 for strip data.

// Segment display paramaters
#define segmentLength 8                               // Segment display "length" of 8
#define segmentDataPin 3                              // D3 for segment display data

#define brightness 100                           // 0 = off, 255 = fullbright

// TLC59711 parameters
#define NUM_TLC59711 1 // amount of chips connected
#define data 8         // PWM data pin (any digital pin)
#define clock 7        // PWM clock pin (any digital pin)
#define ENG_LED 0      // engine LED R0
#define OIL_LED 1      // EOP LED G0
#define DUTY 10        // values from 0 to 100% representing duty cycle

// Engine parameters                                         CHANGE VALUES HERE
#define low_rpm 2200
#define high_rpm 10000
#define WARN_ENG_TEMP 90
#define WARN_EOP 55

// Library Initializations:
MCP_CAN CAN(SPI_CS_PIN);      // Sets CS pin.
Adafruit_NeoPixel strip = Adafruit_NeoPixel(stripLength, stripDataPin, NEO_GRB + NEO_KHZ800);      // Configures the NeoPixel Strip for 16 LEDs.
Adafruit_NeoPixel seg = Adafruit_NeoPixel(segmentLength, segmentDataPin, NEO_GRB + NEO_KHZ800);     // Configures 7-segment display for 8(actually 7) segments

uint16_t PWM_LEVEL = map(DUTY, 0, 100, 0, 65535); // (DUTY/100) * 65535; PWM value ranges from 0 to 65535
Adafruit_TLC59711 LED = Adafruit_TLC59711(NUM_TLC59711, clock, data); // initialization with parameters (chip amount, clock pin, data pin)

**PIN 9** is the chip select pin needed for SPI to communicate with the CAN-BUS Shield. **PIN 10** can be optionally configured to be the chip select pin physically on the CAN-Bus Shield (see shield documentation). **PINs 11-13** are also needed for SPI communication. 16 NeoPixel LEDs are used so it must be defined for the LED strip initialization. **PIN 5** was chosen as the data pin for the LED strip but can be any other free digital pin.

The 7-segment display "has" eight LEDs but the dot has no connection and no LED so there's actually only seven LEDs. **PIN 3** was chosen as the data pin for the segment display but can be any other free digital pin.

Brightness is the initial brightness setting for both NeoPixel strip and segment display but colors can be set to whatever brightness later on.

Number of TLC59711 chips chained together must be defined for initialization. Data and clock pins can be any free digital pins. The PWM driver chip has 12 channels so 12 different chains of LEDs can be controlled. On the chip, pins are labeled RGB with numbers 0-3. Channels are counted up starting from 0. For example, **R0** is channel 0, **G0** is channel 1, **R1** is channel 3, etc. In this case, the engine warning LED is wired to R0 and the EOP LED is wired to G0. A duty percentage is defined for ease of brightness and PWM control.

Engine parameters are defined with the minimum RPM level and maximum RPM shift point for the tachometer. Engine temperature and EOP warnings are set to turn on the PWM LEDs when values exceed the defined values.

mcp_can library is initialized with whatever the chip select pin number is. Typically, and in this case, on **PIN 9**.
The NeoPixel library is initialized with (LED amount, data pin, LED type and timing). Both NeoPixel strip and segment are RGB and have 800kHz data rates. **NEO_GRB + NEO_KHZ800** and others are defined in the .h file of the library.

The PWM driver is 16 bits so it can control PWM signals up to 2^16-1. For ease, the numbers are mapped to 0-100 as the duty cycle. The TLC59711 chip is then initialized with (number of chips chained, clock pin, data pin).

##Variable Definitions

In [ ]:
// Define LED colors, MAY CHANGE BRIGHTNESS
uint32_t greenStrip = strip.Color(0, 20, 0),
         yellowStrip = strip.Color(20, 20, 0),
         redStrip = strip.Color(20, 0, 0),
         redMaxStrip = strip.Color(100, 0, 0),
         blueStrip = strip.Color(0, 0, 20),
         color[4] = {greenStrip, yellowStrip, redStrip, redMaxStrip};

// 7 segment display gear
int digitArray[7][8] = {                       // Gears                      ___________      
                    {0, 0, 1, 0, 1, 0, 1, 0},  // n                         |     0     |
                    {0, 1, 1, 0, 0, 0, 0, 0},  // 1                         |  5     1  |
                    {1, 1, 0, 1, 1, 0, 1, 0},  // 2                         |     6     |
                    {1, 1, 1, 1, 0, 0, 1, 0},  // 3                         |  4     2  |
                    {0, 1, 1, 0, 0, 1, 1, 0},  // 4                         |     3     |
                    {1, 0, 1, 1, 0, 1, 1, 0},  // 5                         |_________7_|
                    {1, 0, 1, 1, 1, 1, 1, 0}   // 6                         
                    };

// 7 segment display startup animation
int digitHello[5][8] = {
                    {0, 1, 1, 0, 1, 1, 1, 0}, //H
                    {1, 0, 0, 1, 1, 1, 1, 0}, //E
                    {0, 0, 0, 1, 1, 1, 0, 0}, //L
                    {0, 0, 0, 1, 1, 1, 0, 0}, //L
                    {1, 1, 1, 1, 1, 1, 0, 0}  //O
                    };

// 7 segment display colors
uint32_t redSeg = seg.Color(255, 0, 0),
         yellowSeg = seg.Color(255, 255, 0),
         greenSeg = seg.Color(0, 255, 0),
         blueSeg = seg.Color(0, 0, 255),
         whiteSeg = seg.Color(255, 255, 255),
         segColor = greenSeg,
         prev_segColor;

// CAN parameters
bool is_CBS_init = false; // True = CAN-BUS initialized succesfully. False = Not initialized yet.

//float LED_RPM;
int shiftPT[2] = {low_rpm, high_rpm};

int prev_range = 10,        // needed for tachometer updating
    prev_gear = 10,         // needed for gear position updating
    ledStages[2] = {5, 11}, // this is where each stage of the led strip is set. i.e. from ledStages[0] and ledStages[1] is stage one and so on
    warningState = 0,       // allows warning to oscillate
    blinkInterval = 50;     // warning flash interval
long prevBlinkTime = 0;     // timer for warning flash interval
bool off = false;           // gear indicator off when engine is off

LED colors are predefined for use on the NeoPixel strip. Colors are defined in RGB format and require an **unsigned 32 bit** variable. RGB values range from 0-255 (R,G,B).

A 2D array is used to hold the predefined gear positions to display on the segment display. Number of rows is for the amount of gears defined, and number of columns is for the eight LEDs on the segment display. On the right in the comments is the segment display pictured with segments associated with the column number. The last column (column 8) should always be 0 since the dot cannot be used.

Other 2D arrays can be defined for custom usage such as startup animations.

As the same with the LED strip, the segment display can use defined colors in RGB format.

Boolean variable to check if the CAN-BUS initialized successfully during intialization. 

Shift point parameters for the tachometer, setting lowest RPM and upper RPM shift warning.

Previous tachometer and gear variables are for updating both only when there's a change to reduce unncessary flickering updates.

Ledstages are to separate the tachometer colors from green to yellow to red depending on the RPM.

WarningState is to keep track of the previous state of the shift warning flash so it can flash on and off.

BlinkInterval is the shift warning flash interval counted in milliseconds with an internal arduino function used as a timer. Requires a **long** variable.